In [13]:
import numpy as np
import pandas as pd
from skimage.feature import blob_log
from laptrack import LapTrack
import napari

# Load images and show in napari

In [14]:
viewer = napari.Viewer()
images = np.load("data/brownian_particles_images_with_noise_small.npz")["images"]
viewer.add_image(images)

<Image layer 'images' at 0x18f823610>

# Detect spots by Laplacian of Gaussian filter

In [15]:
spots = []
for frame, image in enumerate(images):
    _spots = blob_log(image, min_sigma=1, max_sigma=3, num_sigma=3,overlap=0.5)
    df = pd.DataFrame(_spots, columns=["y","x","sigma"])
    df["frame"] = frame
    spots.append(df)
spots_df = pd.concat(spots)
viewer.add_points(spots_df[["frame","y","x"]], size=3, edge_color="yellow", face_color="#ffffff00")

<Points layer 'Points' at 0x17e092c80>

# Track by LapTrack

In [16]:
lt=LapTrack(
    track_cost_cutoff=5**2
)
track_df, _, _ = lt.predict_dataframe(spots_df,["y","x"])
track_df=track_df.reset_index()
viewer.add_tracks(track_df[["track_id","frame","y","x"]])

/Users/fukai/.pyenv/versions/3.9.13/envs/image_analysis/lib/python3.10/site-packages/laptrack/_tracking.py:663: FutureWarning: The parameter only_coordinate_cols will be False by default in the major release.
  warnings.warn(


<Tracks layer 'Tracks' at 0x189406b00>